In [1]:
# importing the required libraries
import pandas as pd
import re
from nltk.tokenize import word_tokenize
import nltk
from singularizer_french import singularize
from re import compile as recompile
nltk.download('punkt')



[nltk_data] Downloading package punkt to /home/talha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# This is a function which reads in the unit file and convert all the data into list so we can use them to change
# the units of ingredient. To add more unit list we have to modify this function and read the additional file of unit
# as well.
# We are lemmatizing and cleaning the file data as well to synchronize it with our processing and so 
#  Mango and mango are not taken as separate ones.
def read_units():
    ml = []
    with open("units_data/ml.txt") as f:
        ml =[singularize(re.sub("\n","",a).strip().lower()) for a in f.readlines() if len(a)>0]
    
    gram = []
    with open("units_data/gram.txt") as f:
        gram = [singularize(re.sub("\n","",a).strip().lower()) for a in f.readlines() if len(a)>0]

    return ml,gram

In [3]:
# There is a case is better understand by an example. We have an ingredient water whose unit we are forcing to be ml.
# Now in one of the ingredient water is coming as 1/2 cups. We simply force it to be 1/2 ml but when we have to add
# this with other ml of water like we might have 2 ml water, adding 1/2 and 2 is not possible in python as 1/2 is 
# consider a string. So we need to convert 1/2 into 0.5. so this function does that.
def divider(number):
    a_q = number.split("/")
    a1 = int(a_q[0])
    a2 = int(a_q[1])
    return a1/a2 


In [4]:
# This is the function which covnerts the ingredient unit according to the file we have created. This function recieves
# those list as shown by ml and gram parameter and they also takes the ingredient name, unit and quantity.
# The major functionality is to check the name and if the name lies in the list, it converts it. The issue was
#  occuring that if the ingredient is in l we should convert that to kg and not g so we need to check that as well
# and because of that we have the quantity with us as well.
def convert_unit(substring4,unit,quantity_product,ml,gram,m):
    for m in ml:
        if m in substring4 and len(m)>0 and unit !="l":
            try:
                if unit == "kg":
                    return quantity_product,"l"
                else:
                    return quantity_product/1000,"l"
            except:
                return divider(quantity_product)/1000,"l"
    for g in gram:
        if g in substring4 and len(g)>0 and unit !="kg":
            try:
                if unit == "l":
                    return quantity_product,"kg"
                else:
                    return quantity_product/1000,"kg"
            except:
                return divider(quantity_product)/1000,"kg"
            
    return quantity_product,unit

In [5]:
# The pattern thats found in the ingredient listing is that words after a comma is mostly attributes of ingredient
# which is not required by us. so we remove that by spliting. This case fails few time and we hard coded them because
#  we where not finding any good pos tagger for french
def comma_separator(m):
    split_string1 = m.split(',')
    if ", " in m:
        if split_string1[1].strip() == "sans peau":

            substring2 = split_string1[0] +", "+split_string1[1]
        else:
            substring2 = split_string1[0]         
    else:
        substring2 = m 
    if split_string1[0] == "6 tranches ":
        substring2 = split_string1[1]
        
    return substring2

In [6]:
# This is same as comma separator, we didn't require the text which comes after. There are no pattern when this breaks
def or_separator(substring2):
    split_string = substring2.split(' ou ')
    substring1 = ""
    substring1 = split_string[0]
    
    return substring1

In [7]:
# Main function that processes all and return the dataframe having all ingredients with there quantity and unit.
# it takes in the json data with the ml and gram unit lists as well. 
def process(input_json_data,ml,gram):

    data = input_json_data
    df = None
    des,quantity,unit = [],[],[]
    
    #     THe one gram words we want to remove from the text
    unknow_words = ["nature","jeune","cuillères","cuillère","à","thé","c","en","dés","hachée","hachées","grossièrement","cuillère","divisées","finement","facultatif","haché","tassées","boîte","soupe","conserve","bouillon","gousses","gousse","grosse","gross","gros","lanières","livre","onces","paquet","petit","petite","récipients","sachet","tasse","tasses","râpé","non cuit","cuit","cuits","tranche","tranches","tranch","moyenne","durs","pincer","refroidi","refroidie","grande","pot","emballée","poche","paquet","ordinaire","bandes","chaude","chaudes","plus","coupées","lanières","moitié","cuillères à thé","en","cubes","cube","dénoyauté","dénoyautés","mince","minces","tout petit","pelée","pelées","rôti","rôtis",]
    #     THe units we may encounter in our ingredients
    units = [" onces "," ml ", " l "," g "," kg "," kgs "," oz "," cm "," tasse de "]
    #     The other small things and issues we need to remove which where no useful
    units_remove = ['%','/','.','-','_','-',']','*','-/', 'c.',"[,","]","onces","ml", "l","g","kg","kgs","oz","cm"]
#     looping over the data
    for result in data['data']:
        #         Picking all the ingredients in a reciepe and then we loop over it as well
        d= result[u'ingredients'][u'non classés'][u'list']
        for a in d:
#             Storing the ingredient
            m= a['description']
#          there was issue in quantity listing where the quantity was 5,6 which needed to be corrected and we did that here
            rgx = recompile(r'(?<=\d)[,](?=\d)')
            m = rgx.sub(".",m)
#             m =re.sub("d'","de ",m)
#             Tokenization means converting the sentence into words
            word_tokens = word_tokenize(m) 
#             Then re joining them to form a sentence again. The usefulness of this step is that it will add 
#             spaces with every word and punctuation
            m  = (" ").join(word_tokens)
            m = re.sub("\. "," ",m)
            

#         calling the comma separator                
            substring2 = comma_separator(m)
#     calling the or separator
            substring1 = or_separator(substring2)
#     Lowering the text so English and english becomes same and not different
            substring1 = substring1.lower()

    
#     This portion is for deciding the unit of ingredient.it has multiple steps to check the units
#     it checks if any of the unit exists in the ingredient from the list we created above. The last unit is 
#     preferred over everybody else. If the unit within string and in attributes matches that unit is finalized
#     without looking for other units
#     if no unit was found the attribute unit is assigned and even if that is empty x is assigned to it
            index = 0
            u = "x"
            for x in units:
                finding = max(substring1.find(x),0)
                if finding > index:
                  index =finding
                  u = x
            if u == "x" and a['unit'] != None:
                u = a['unit']
            
# From here the quantity extraction process starts. We want the quantity that comes before our selected unit.
# To do that we do the pattern match and pick the number before the unit.
# there are different ways how a number can be listed so we need to check pattern for all.
#  1/2 , 0/5,4 are different styles of quantity.

# After checking the pattern we extract that quantity and is stored. If no quantity is found we go to the quantity
# attribute
            s = substring1
            pattern1 = r"(\d+){}".format(u)
            pattern2 = r"(\d+/\d+){}".format(u)
            pattern3 = r"(\d+.\d+){}".format(u)

            if len(re.findall(pattern2, s))>0:
                quantity_product = re.findall(pattern2, s)[0]

            elif len(re.findall(pattern1, s))>0:
                if len(re.findall(pattern3, s))>0:
                    quantity_product = re.findall(pattern3, s)[0]

                else:
                    quantity_product = re.findall(pattern1, s)[0]
            else:
                quantity_product = a['quantity']
                
            if index !=0:
                index = index + len(u.strip())+1

# removing the brackets and text inside brackets
            substring3 = substring1
            substring3 = re.sub("\(.*?\)","",substring3)
            
            index = [i for i in range(len(substring3)) if substring3.startswith("(", i)]

            if len(index) == 0:
                pass
            else:
                index = index[0]
                substring3 = substring3[:index]  


            index = [i for i in range(len(substring3)) if substring3.startswith(")", i)]

            if len(index) == 0:
                pass
            else:
                index = index[-1]
                substring3 = substring3[index+1:]      
            x = re.findall("\d", substring3)
# here we are removing all extra numbers inside an ingredient 
            for aaa in set(x):
                substring3 = re.sub(aaa,"",substring3)
            substring4 = substring3
            word_tokens = word_tokenize(substring4) 
            
            
#             tokenizing and removing all bigrams word according to the list we created above. We are also doing
# lemmatization which singularize the words
#             substring4 = singularize(substring4)
            filtered_sentence = [singularize(w) for w in word_tokens if not w.strip() in unknow_words+units_remove] 
            substring4  = (" ").join(filtered_sentence)

#             basic striping and cleaning
            substring4 = substring4.strip(",")
            substring4 = substring4.strip()
            substring4 = re.sub("-","",substring4)

#       now the portion for quantity conversion starts. We wanted to convert all the different variants of liters and
#      kgs to be one so we do that here. if its in ml or g we also divide it by 1000 to make it equal to kg
            u = u.strip()
            try:
                quantity_product =float(quantity_product)
                if u == 'litres' or u == 'litre' or u == "liter" or u == "liters" or u == "ml" or u == "l":
                    if u == "ml":
                        quantity_product = quantity_product/1000
                    u="l"
                elif u == "kgs" or u == "kg" or u == "g":
                    if u == "g":
                        quantity_product = quantity_product/1000
                    u="kg"
            except:
                pass

#             converting the unit. first we synchronize the unit one step back and now we want to convert them.
#          we simply call the function we created with the list and it will get us the results
            output_converter = convert_unit(substring4,u,quantity_product,ml,gram,m)

# here we are dealing with de and d'. If they are coming in start or end we don't want them but if they are in middle
# we do need them so we are solving that here.
            quantity_product,u = output_converter[0],output_converter[1]
            substring4 = re.sub("de ","",substring4)
            substring4 = re.sub("^[\s]*d'","",substring4)

#         now after everything is processing we convert them to form a dataframe and return the info for further process
            des.append(substring4)
            quantity.append(quantity_product)
            unit.append(u)
            df =pd.DataFrame(zip(des,quantity,unit),columns=["food","quantity","unit"])

    return df

In [8]:
# reading all the category files cleaning them and creating there lists for checking
def create_category_list():
  bakery = []
  with open("categories_data/Boulangerie.txt") as f:
    bakery = [add_spaces(singularize(re.sub("\n","",a).strip())) for a in f.readlines() if len(a)>0]

  chilled = []
  with open("categories_data/Chilled_2.txt") as f:
    chilled = [add_spaces(singularize(re.sub("\n","",a).strip())) for a in f.readlines() if len(a)>0]

  beverages = []
  with open("categories_data/Breuvages.txt") as f:
    beverages = [add_spaces(singularize(re.sub("\n","",a).strip())) for a in f.readlines() if len(a)>0]

  dairy = []
  with open("categories_data/produits_laitiers_et_œufs.txt") as f:
    dairy = [add_spaces(singularize(re.sub("\n","",a).strip())) for a in f.readlines() if len(a)>0]

  fruit = []
  with open("categories_data/fruits_et_légumes.txt") as f:
    fruit = [add_spaces(singularize(re.sub("\n","",a).strip())) for a in f.readlines() if len(a)>0]

  grains = []
  with open("categories_data/céréales_et_haricots.txt") as f:
    grains = [add_spaces(singularize(re.sub("\n","",a).strip())) for a in f.readlines() if len(a)>0]

  herbs = []
  with open("categories_data/herbes_et_épices.txt") as f:
    herbs = [add_spaces(singularize(re.sub("\n","",a).strip())) for a in f.readlines() if len(a)>0]

  meat = []
  with open("categories_data/viande_poisson_et_substituts.txt") as f:
    meat = [add_spaces(singularize(re.sub("\n","",a).strip())) for a in f.readlines() if len(a)>0]
    
  nuts = []
  with open("categories_data/noix_et_graines.txt") as f:
    nuts = [add_spaces(singularize(re.sub("\n","",a).strip())) for a in f.readlines() if len(a)>0]
    
  pantry = []
  with open("categories_data/garde-manger.txt") as f:
    pantry = [add_spaces(singularize(re.sub("\n","",a).strip())) for a in f.readlines() if len(a)>0]

  return bakery,chilled,beverages, dairy, fruit,grains,herbs,meat,nuts,pantry

In [9]:
# the function adds spaces before and after a word. This was required due to certain issues.
# eg we want to find the category of rice which is listed in I think pantry. We also have ice listed in chilled. 
# now when we try to match ice with rice it give 100% match as ice is there in rice and we dont want that. so we added
# spaces this covnerted the "ice" into " ice " which is not present in "rice "
def add_spaces(text):
    return " "+text+" "

In [10]:
# this function takes in all the list and the ingredient name and return its category
def name_cat(name,bakery,chilled,beverages, dairy, fruit,grains,herbs,meat,nuts,pantry):
  
  for a in bakery:
    if a in name and len(a)>0:
      return 0

  for a in chilled:
    if a.lower() in name.lower() and len(a)>0:
      return 1

  for a in beverages:
    if a.lower() in name.lower() and len(a)>0:
      return 2

  for a in dairy:
    if a.lower() in name.lower() and len(a)>0:
      return 3

  for a in fruit:
    if a.lower() in name.lower() and len(a)>0:
      return 4

  for a in grains:
    if a.lower() in name.lower() and len(a)>0:
      return 5

  for a in herbs:
    if a.lower() in name.lower() and len(a)>0:
      return 6

  for a in meat:
    if a.lower() in name.lower() and len(a)>0:
      return 7

  for a in nuts:
    if a.lower() in name.lower() and len(a)>0:
      return 8

  for a in pantry:
    if a.lower() in name.lower() and len(a)>0:
      return 9

  return 10

In [11]:
# the main function that calls all the functions and synchronize everything. It only takes in the data
def main_function(input_json):
    #     Calling the function which reads the unit files
    ml,gram = read_units()
    #     doing the main process and getting the dataframe
    df = process(input_json,ml,gram)
    #     grouping them as to calculate the total quantity of each unique product
    df1 = df.groupby(["food","unit"])
    #     reading all the unit files
    bakery,chilled,beverages, dairy, fruit,grains,herbs,meat,nuts,pantry = create_category_list()
    #     creating lists to store the ingredients under each category
    json_bakery = []
    json_chilled = []
    json_beverages=[] 
    json_dairy =[] 
    json_fruit=[]
    json_grains =[] 
    json_herbs = []
    json_meat =[]
    json_nuts =[]
    json_pantry =[]
    json_others = []
    #     looping over the grouped dataframe to clean and store it.
    for name,group in df1:
        new = {}
        new["ingredient"]=name[0].strip()
#         this logic supports the idea that if we have quantity less then 1 l it should be in ml. so this code snippet
# does that with kg and l. it convertes the quantity and unit if less then 1. It then round it to 2 digits. 
# it also supports the idea that if we have a quantity 2.99999, it should get 3 rather then 2.99.
        try:
            if name[1] == "l" and group["quantity"].sum()<1:
                new["unit"]="ml"
                new["quantity"]=round(float(group["quantity"].sum()*1000),2)
            elif name[1] =="kg" and group["quantity"].sum()<1:
                new["unit"]="g"
                new["quantity"]=round(float(group["quantity"].sum()*1000),2)
            else:
                new["unit"]=name[1]
                if round(group["quantity"].sum(),2) == int(group["quantity"].sum())+0.99:
                    new["quantity"] = round(group["quantity"].sum(),0)
                else:
                    new["quantity"]=round(group["quantity"].sum(),2)
        except:
            #             we were having issues with quantity 1/2 so this code deal with that.
            new["unit"]=name[1]
            total = 0
            
            for qw in group["quantity"]:
                a = qw.split("/")
                
                a1 = int(a[0])
                a2 = int(a[1])
                total = total + a1/a2
                
            if round(total,2) == 0.33:
                quantity = "1/3"
            else:
                out = (total).as_integer_ratio()
                quantity = str(out[0])+"/"+str(out[1])
                if out[0] == out[1]:
                    quantity = out[0]
            
            new["quantity"] = quantity
            # getting the category name by calling the function
        cat = name_cat(add_spaces(name[0]),bakery,chilled,beverages, dairy, fruit,grains,herbs,meat,nuts,pantry) 
#         according to category name adding the info in that list created above
        if cat == 0:
            json_bakery.append(new)
        elif cat == 1:
            json_chilled.append(new)
        elif cat == 2:
            json_beverages.append(new)
        elif cat == 3:
            json_dairy.append(new)
        elif cat == 4:
            json_fruit.append(new)
        elif cat == 5:
            json_grains.append(new)
        elif cat == 6:
            json_herbs.append(new)
        elif cat == 7:
            json_meat.append(new)
        elif cat == 8:
            json_nuts.append(new)
        elif cat == 9:
            json_pantry.append(new)
        elif cat == 10:
            json_others.append(new)

# sending the final output that is recieved by the front end. 
    final = {"garde manger":json_pantry,"Breuvages":json_beverages,"fruits et légumes":json_fruit,
            "viande poisson et substituts":json_meat,"produits laitiers et œufs":json_dairy,
             "Chilled":json_chilled,"céréales et haricots":json_grains,"herbes et épices":json_herbs,
            "noix et graines":json_nuts,"Boulangerie":json_bakery,"Vous pourriez également avoir besoin":json_others}
    
    
    return final


In [12]:

import json
f = open('total_french.json',encoding="utf-8")
data = json.load(f)
aa = main_function(data)

In [13]:
with open("sample.json", "w",encoding="utf-8") as outfile: 
    json.dump(aa, outfile,indent=4,ensure_ascii=False)